In [12]:
#Import required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Data Import

In [13]:
#Import CSV file as dataframe
data_df = pd.read_csv("canine_cushings_disease_clean.csv")

#Set PatientID as index
data_df.index = data_df['PatientID']

#Remove PatientID column
del data_df['PatientID']

#Check df
data_df.head()

,Diagnosis,BreedID,Breed,Age,Sex,Neutered,WeightChange,Weight,Hospitalised,HeartRate,...,Polydipsia,Polyuria,IncreasedALKP,IncreasedATL/GGT/AST,LowUSG,Proteinuria,Hypertensive,LowBloodSugar,Hypothyroid,CruciateSurgery
PatientID,,,,,,,,,,,,,,,,,,,,,
1,1,15009.0,Jack Russell Terrier,12,M,True,False,13.00,False,NaN,...,True,True,True,True,NaN,True,False,False,False,False
2,1,13714.0,Bichon Frise,12,F,True,False,9.30,False,NaN,...,True,True,True,True,True,True,False,False,False,False
3,1,15086.0,West Highland White Terrier,13,F,True,No change,9.82,False,NaN,...,True,True,True,True,False,True,False,False,False,False
4,1,14846.0,Cocker Spaniel,12,F,True,False,11.30,True,120.0,...,True,True,NaN,NaN,NaN,NaN,False,False,False,False
5,1,99999.0,Crossbreed,9,F,False,False,9.30,False,96.0,...,True,True,True,True,True,True,False,False,False,False


### Data Cleaning

In [14]:
#Check data types
data_df.dtypes

Diagnosis                  int64
BreedID                  float64
Breed                     object
Age                        int64
Sex                       object
Neutered                    bool
WeightChange              object
Weight                   float64
Hospitalised                bool
HeartRate                float64
Temperature              float64
Vomiting                    bool
Diarrhoea                   bool
Hepatomegaly                bool
ThinDrySkin                 bool
Alopecia                    bool
Pruritus                    bool
MuscleWastageWeakness       bool
Lethargy                    bool
Panting                     bool
Neurological                bool
Polydipsia                  bool
Polyuria                    bool
IncreasedALKP             object
IncreasedATL/GGT/AST      object
LowUSG                    object
Proteinuria               object
Hypertensive                bool
LowBloodSugar               bool
Hypothyroid                 bool
CruciateSu

In [15]:
#Change BreedID to object data type
data_df['BreedID'] = data_df['BreedID'].astype(object)

#Change Diagnosis to object data type
data_df['Diagnosis'] = data_df['Diagnosis'].astype(object)

#Check data types
data_df.dtypes

Diagnosis                 object
BreedID                   object
Breed                     object
Age                        int64
Sex                       object
Neutered                    bool
WeightChange              object
Weight                   float64
Hospitalised                bool
HeartRate                float64
Temperature              float64
Vomiting                    bool
Diarrhoea                   bool
Hepatomegaly                bool
ThinDrySkin                 bool
Alopecia                    bool
Pruritus                    bool
MuscleWastageWeakness       bool
Lethargy                    bool
Panting                     bool
Neurological                bool
Polydipsia                  bool
Polyuria                    bool
IncreasedALKP             object
IncreasedATL/GGT/AST      object
LowUSG                    object
Proteinuria               object
Hypertensive                bool
LowBloodSugar               bool
Hypothyroid                 bool
CruciateSu

### Scaling Feature Values